# 1 - Imports and functions plotting

In [1]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

In [2]:
mesh1 = TriMesh('data/cat-00.off')
mesh2 = TriMesh('data/lion-00.off')
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 7207 vertices, 14410 faces
Mesh 2 : 5000 vertices,  9996 faces


In [3]:
k1,k2 = 50,50 # Number of eigenvalues on source and Target
landmarks = np.loadtxt('data/landmarks.txt',dtype=int)[:5] # loading 5 landmarks

In [4]:
model = FunctionalMapping(mesh1,mesh2)
model.preprocess(n_ev=(k1,k2),subsample_step = 5, landmarks = landmarks, verbose=True)


Computing Laplacian spectrum
Compute linear FEM Laplace-Beltrami spectrum
Compute linear FEM Laplace-Beltrami spectrum

Computing descriptors
	Normalizing descriptors

	120 out of 600 possible descriptors kept
	Done


In [5]:
descr_mu = 1e0
lap_mu = 1e-3
descr_comm_mu=1e-1

model.fit(descr_mu = descr_mu,lap_mu = lap_mu, descr_comm_mu=descr_comm_mu, verbose=True)

Computing new descriptors
	Done

Optimization :
	50 Ev on source - 50 Ev on Target
	Using 120 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-03

	Task : b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH', funcall : 53, nit : 50, warnflag : 0
	Done


In [6]:
p2p = model.p2p
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p]
double_plot(mesh1,mesh2,cmap1,cmap2)

In [7]:
model.icp_refine()
p2p = model.p2p
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p]
double_plot(mesh1,mesh2,cmap1,cmap2)

In [9]:
model.change_FM_type('classic')
model.zoomout_refine(nit=15)
p2p = model.p2p
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p]
double_plot(mesh1,mesh2,cmap1,cmap2)